# Evaluación y Comparativa de Modelos de Incumplimiento
Este notebook se enfoca en la construcción, evaluación y comparación de diversos algoritmos para predecir la probabilidad de default. El objetivo es seleccionar el modelo que mejor logre identificar el riesgo manteniendo un equilibrio con los objetivos de negocio.

## 1. Configuración Inicial y Carga de Datos
Importamos las librerías necesarias para el procesamiento, modelado y visualización. Cargamos la base de datos ya filtrada y limpia.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# Preprocessing
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Métricas
from sklearn.metrics import (roc_auc_score, roc_curve, confusion_matrix,
                             classification_report, precision_recall_curve)


In [3]:

apps_model = pd.read_csv('data/clean/apps_model_filtered.csv', parse_dates=['orig_month'], thousands=',')
apps_model

,customer_id,orig_month,age,income,debt_income,bureau_score,utilization,prev_delin_24m,tenure_months,amount,...,rate,infl,default_12m,months_since_orig,product_CC,product_PL,channel_Digital,channel_Partner,region_N,region_S
0,2,2024-01-03,34,103244.98,0.3936,691.9,0.1771,1,17,45248.85,...,7.375842,4.507775,0,12.155059,0,1,0,1,0,0
1,3,2024-01-05,31,41568.33,0.5023,653.2,0.4781,0,68,308801.45,...,7.384582,4.868222,1,12.089356,0,0,1,0,0,0
2,5,2024-01-01,50,12633.20,0.1435,681.7,0.1931,0,40,92093.10,...,7.382953,4.249541,0,12.220762,0,1,1,0,0,0
3,8,2022-01-10,29,24126.21,0.3842,603.6,0.2423,1,17,84014.16,...,6.908256,4.278983,0,35.906702,0,1,1,0,1,0
4,9,2022-01-03,21,34684.79,0.4979,702.9,0.0808,0,81,22040.43,...,6.886504,4.319146,0,36.136662,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18421,24993,2022-01-04,57,30805.18,0.3133,623.7,0.6330,0,82,38923.00,...,7.088503,4.285473,1,36.103811,0,1,1,0,0,1
18422,24995,2024-01-04,47,37852.79,0.2701,668.7,0.4662,0,50,69841.34,...,7.176895,4.608903,0,12.122208,0,1,1,0,0,0
18423,24996,2023-01-07,38,30300.98,0.2434,721.2,0.6266,1,3,46228.76,...,7.137931,4.360457,0,24.014455,1,0,0,1,0,1
18424,24998,2024-01-06,50,19879.46,0.4391,533.4,0.3975,0,15,33748.71,...,7.444711,4.886401,0,12.056505,0,1,1,0,0,1


## 2. Preparación de Muestras (Out-of-Time Split)
Para garantizar que el modelo sea capaz de predecir el futuro, dividimos los datos cronológicamente. Usamos el 80% más antiguo para entrenamiento y el 20% más reciente como muestra de prueba (Out-of-Time).

In [4]:
apps_sorted = apps_model.sort_values('orig_month')


In [5]:
cutoff = apps_sorted['orig_month'].quantile(0.8)

train = apps_sorted[apps_sorted['orig_month'] <= cutoff]
test  = apps_sorted[apps_sorted['orig_month'] > cutoff]


## 3. Definición de Variables Predictoras (Features)
Identificamos las variables que entrarán al modelo, excluyendo identificadores y la variable objetivo para evitar fugas de información.

In [ ]:
features = apps_sorted.columns.difference(['customer_id','orig_month','default_12m'])

X_train = train[features]
y_train = train['default_12m']

X_test = test[features]
y_test = test['default_12m']



## 4. Normalización de Datos
Estandarizamos las variables para que tengan media 0 y desviación estándar 1. Esto permite que modelos como la Regresión Logística traten a todas las variables por igual, independientemente de su unidad de medida original.

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

scaler = StandardScaler()

X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)


## 5. Diagnóstico de Multicolinealidad
Analizamos la correlación entre las variables. Una correlación muy alta entre dos predictores puede inflar la varianza de los coeficientes y dificultar la interpretación.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Calculamos la matriz de correlación del set escalado
corr_matrix = X_train_scaled.corr()

# 2. Configuramos el tamaño de la figura
plt.figure(figsize=(12, 10))

# 3. Creamos el mapa de calor
sns.heatmap(
    corr_matrix,
    annot=True,          # Muestra los números en cada celda
    fmt=".2f",           # Limita a 2 decimales
    cmap='coolwarm',     # Color: azul (negativo), rojo (positivo)
    linewidths=0.5,      # Espacio entre celdas
    center=0             # El blanco será el valor 0 (sin correlación)
)

plt.title('Mapa de Calor de Correlaciones')
plt.show()

## 6. Regresión Logística: Modelo de Referencia
Iniciamos con una Regresión Logística estándar utilizando todas las variables disponibles para observar su significancia estadística (p-values).

In [ ]:
import statsmodels.api as sm

X_train_sm = sm.add_constant(X_train_scaled)
logit_model = sm.Logit(y_train, X_train_sm)
result = logit_model.fit()

print(result.summary())

## 7. Selección de Variables y Modelo Reducido
Seleccionamos las variables más representativas basándonos en criterios estadísticos y de negocio. Esto simplifica el modelo, haciéndolo más robusto y fácil de implementar.

In [ ]:
good_vars = [
    'bureau_score',
    'debt_income',
    'income',
    'infl',
    'prev_delin_24m',
    'rate',
    'utilization'
]
X_train_red = X_train[good_vars]
X_test_red  = X_test[good_vars]
scaler = StandardScaler()

X_train_red_scaled = pd.DataFrame(
    scaler.fit_transform(X_train_red),
    columns=good_vars,
    index=X_train_red.index
)

X_test_red_scaled = pd.DataFrame(
    scaler.transform(X_test_red),
    columns=good_vars,
    index=X_test_red.index
)
X_test_red_scaled

In [ ]:
X_train_sm_red = sm.add_constant(X_train_red_scaled)

logit_red = sm.Logit(y_train, X_train_sm_red)
result_red = logit_red.fit()

print(result_red.summary())

In [ ]:
comparacion = pd.DataFrame({
    'Modelo': ['Completo (19 vars)', 'Reducido (7 vars)'],
    'Pseudo R': [result.prsquared, result_red.prsquared],
    'AIC': [result.aic, result_red.aic],
    'BIC': [result.bic, result_red.bic],
    'Log-Likelihood': [result.llf, result_red.llf]
})
print(comparacion)


In [ ]:
X_test_sm_red = sm.add_constant(X_test_red_scaled)
y_pred_proba = result_red.predict(X_test_sm_red)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

auc = roc_auc_score(y_test, y_pred_proba)
print("AUC:", auc)


## 8. Regresión Logística Balanceada
Entrenamos el modelo ajustando los pesos de las clases. Dado que el default es un evento poco frecuente, este ajuste ayuda a que el algoritmo no ignore a los clientes riesgosos.

In [ ]:
from sklearn.linear_model import LogisticRegression

# Con scikit-learn (más simple)
logit_balanced = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42
)

logit_balanced.fit(X_train_red_scaled, y_train)
y_pred_logit_balanced = logit_balanced.predict_proba(X_test_red_scaled)[:, 1]

# Métricas Logit
auc_logit = roc_auc_score(y_test, y_pred_logit_balanced)
gini_logit = 2 * auc_logit - 1

# KS Logit
scores_logit_0 = y_pred_logit_balanced[y_test == 0]
scores_logit_1 = y_pred_logit_balanced[y_test == 1]
ks_logit = stats.ks_2samp(scores_logit_0, scores_logit_1).statistic

print(f"AUC:  {auc_logit:.4f}")
print(f"Gini: {gini_logit:.4f}")
print(f"KS:   {ks_logit:.4f}")

## 9. Algoritmos de Potenciación de Gradiente (XGBoost)
Probamos XGBoost, un algoritmo basado en árboles de decisión que captura relaciones complejas y no lineales. Iniciamos con una versión base sin ajustes de balance.

In [ ]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score

# Usa las 19 variables originales SIN escalar
# (XGBoost no requiere escalado)
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    random_state=42,
    eval_metric='auc'
)

xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict_proba(X_test)[:, 1]

# Métricas XGBoost Baseline
auc_xgb = roc_auc_score(y_test, y_pred_xgb)
gini_xgb = 2 * auc_xgb - 1

# KS XGBoost Baseline
scores_xgb_0 = y_pred_xgb[y_test == 0]
scores_xgb_1 = y_pred_xgb[y_test == 1]
ks_xgb = stats.ks_2samp(scores_xgb_0, scores_xgb_1).statistic

print(f"AUC:  {auc_xgb:.4f}")
print(f"Gini: {gini_xgb:.4f}")
print(f"KS:   {ks_xgb:.4f}")

## 10. XGBoost con Ajuste de Pesos
Configuramos XGBoost para que asigne más importancia a los casos de default, buscando mejorar la detección de riesgo en una población desbalanceada.

In [ ]:
# Seccion de XGBoost Balanced
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()
ratio = neg / pos

xgb_balanced = xgb.XGBClassifier(
    scale_pos_weight=ratio,
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=3,
    gamma=0.1,
    reg_alpha=0.1,
    reg_lambda=1,
    random_state=42,
    eval_metric='auc'
)

xgb_balanced.fit(X_train, y_train)
y_pred_balanced = xgb_balanced.predict_proba(X_test)[:, 1]

# Métricas XGBoost Balanced
auc_balanced = roc_auc_score(y_test, y_pred_balanced)
gini_balanced = 2 * auc_balanced - 1

# KS XGBoost Balanced
scores_balanced_0 = y_pred_balanced[y_test == 0]
scores_balanced_1 = y_pred_balanced[y_test == 1]
ks_balanced = stats.ks_2samp(scores_balanced_0, scores_balanced_1).statistic

print(f"Ratio de desbalance: {ratio:.2f}")
print(f"AUC:  {auc_balanced:.4f}")
print(f"Gini: {gini_balanced:.4f}")
print(f"KS:   {ks_balanced:.4f}")


## 11. Análisis de Umbrales de Decisión
Exploramos cómo varía la clasificación al cambiar el punto de corte (threshold). Esto permite ajustar el modelo según el apetito de riesgo de la institución.

In [ ]:
y_pred_balanced = xgb_balanced.predict_proba(X_test)[:, 1]


nuevo_umbral = 0.20  # Ejemplo: Si la probabilidad es > 20%, clasifícalo como Default

# Creamos las predicciones binarias (Clases 0 y 1) con ese umbral
y_pred_clase_ajustada = (y_pred_balanced >= nuevo_umbral).astype(int)

# ---------------------------------

# Ahora calculas la matriz de confusión con esta nueva variable
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_clase_ajustada)
print(f"Matriz de Confusión con umbral {nuevo_umbral}:")
print(cm)

## 12. XGBoost con Sobremuestreo Sintético (SMOTE)
Aplicamos la técnica SMOTE para generar casos sintéticos de default y equilibrar artificialmente la muestra antes de entrenar el modelo.

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42, k_neighbors=5)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

xgb_smote = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=3,
    gamma=0.1,
    random_state=42,
    eval_metric='auc'
)

xgb_smote.fit(X_train_smote, y_train_smote)
y_pred_smote = xgb_smote.predict_proba(X_test)[:, 1]
# Métricas XGBoost SMOTE
auc_smote = roc_auc_score(y_test, y_pred_smote)
gini_smote = 2 * auc_smote - 1

# KS XGBoost SMOTE
scores_smote_0 = y_pred_smote[y_test == 0]
scores_smote_1 = y_pred_smote[y_test == 1]
ks_smote = stats.ks_2samp(scores_smote_0, scores_smote_1).statistic

print(f"AUC:  {auc_smote:.4f}")
print(f"Gini: {gini_smote:.4f}")
print(f"KS:   {ks_smote:.4f}")

## 13. Bosques Aleatorios (Random Forest)
Entrenamos un modelo de Random Forest con pesos balanceados. Este algoritmo es conocido por su estabilidad y resistencia al sobreajuste.

In [ ]:
rf_balanced = RandomForestClassifier(
    n_estimators=300,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=0
)

rf_balanced.fit(X_train, y_train)
y_pred_rf = rf_balanced.predict_proba(X_test)[:, 1]

# Métricas Random Forest
auc_rf = roc_auc_score(y_test, y_pred_rf)
gini_rf = 2 * auc_rf - 1

# KS Random Forest
scores_rf_0 = y_pred_rf[y_test == 0]
scores_rf_1 = y_pred_rf[y_test == 1]
ks_rf = stats.ks_2samp(scores_rf_0, scores_rf_1).statistic

print(f"AUC:  {auc_rf:.4f}")
print(f"Gini: {gini_rf:.4f}")
print(f"KS:   {ks_rf:.4f}")

## 14. Comparativa General de Desempeño
Contrastamos todos los modelos bajo las tres métricas principales de riesgo:
- **AUC:** Capacidad general de separación.
- **Gini:** Eficiencia en el ordenamiento del riesgo.
- **KS:** Máxima separación entre clientes buenos y malos.

In [ ]:
# Lista maestra de modelos
models = [
    ('XGB Baseline', y_pred_xgb, 'gray'),
    ('XGB Balanced', y_pred_balanced, 'blue'),
    ('XGB SMOTE', y_pred_smote, 'green'),
    ('Random Forest', y_pred_rf, 'purple'),
    ('Logit Balanced', y_pred_logit_balanced, 'orange')
]

# Variables para los gráficos de barras
model_names = ['XGB\nBaseline', 'XGB\nBalanced', 'XGB\nSMOTE', 'Random\nForest', 'Logit\nBalanced']
colors_bar = ['gray', 'blue', 'green', 'purple', 'orange']

# Listas de valores (asegúrate de que las variables gini_* y ks_* existan)
gini_values = [gini_xgb, gini_balanced, gini_smote, gini_rf, gini_logit]
ks_values = [ks_xgb, ks_balanced, ks_smote, ks_rf, ks_logit]

## 15. Visualización Comparativa
Graficamos los resultados para identificar visualmente cuál modelo ofrece el mejor desempeño en la muestra Out-of-Time.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

for name, y_pred, color in models:
    fpr, tpr, _ = roc_curve(y_test, y_pred)
    auc_val = roc_auc_score(y_test, y_pred)
    ax.plot(fpr, tpr, label=f'{name} (AUC: {auc_val:.3f})', color=color, linewidth=2)

ax.plot([0, 1], [0, 1], '--', color='red', label='Random (0.500)')
ax.set_xlabel('False Positive Rate', fontsize=11)
ax.set_ylabel('True Positive Rate', fontsize=11)
ax.set_title('ROC Curves Comparison', fontsize=12, fontweight='bold')
ax.legend(fontsize=9, loc='lower right')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

### Comparativa de Gini y KS
Estas métricas son fundamentales en el sector financiero para determinar qué tan bien el modelo segmenta a la población según su nivel de riesgo.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

ax.barh(model_names, gini_values, color=colors_bar, alpha=0.7)
ax.set_xlabel('Gini Coefficient', fontsize=11)
ax.set_title('Gini Index Comparison', fontsize=12, fontweight='bold')
ax.grid(alpha=0.3, axis='x')

for i, val in enumerate(gini_values):
    ax.text(val + 0.01, i, f'{val:.4f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

### 3. Estadística KS (Kolmogorov-Smirnov)
Mide la máxima separación entre las funciones de distribución acumulada de los clientes que caen en default y los que no.


In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

ax.barh(model_names, ks_values, color=colors_bar, alpha=0.7)
ax.set_xlabel('KS Statistic', fontsize=11)
ax.set_title('KS Statistic Comparison', fontsize=12, fontweight='bold')
ax.grid(alpha=0.3, axis='x')

for i, val in enumerate(ks_values):
    ax.text(val + 0.01, i, f'{val:.4f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

### Resumen Ejecutivo de Métricas

In [ ]:
import pandas as pd

# Crear tabla resumen
metricas_resumen = pd.DataFrame({
    'Modelo': ['XGB Baseline', 'XGB Balanced', 'XGB SMOTE', 'Random Forest', 'Logit Balanced'],
    'AUC': [auc_xgb, auc_balanced, auc_smote, auc_rf, auc_logit],
    'Gini': [gini_xgb, gini_balanced, gini_smote, gini_rf, gini_logit],
    'KS': [ks_xgb, ks_balanced, ks_smote, ks_rf, ks_logit]
})

metricas_resumen = metricas_resumen.sort_values('AUC', ascending=False)

print("\n" + "=" * 60)
print("RESUMEN DE MÉTRICAS - TODOS LOS MODELOS")
print("=" * 60)
print(metricas_resumen.to_string(index=False, float_format="%.4f"))

### Análisis de Matrices de Confusión
Detallamos los aciertos y errores (Falsos Positivos y Falsos Negativos) para entender el impacto operativo de cada modelo.

In [ ]:
import matplotlib.pyplot as plt

# Definimos el umbral estándar para la clasificación
threshold = 0.5

# Lista de configuración para iterar (Nombre, Predicciones, Paleta de colores)
# Asegúrate de que las variables y_pred_* existen en tu memoria
models_cm = [
    ('Logit Balanced', y_pred_logit_balanced, 'Oranges'),
    ('XGB Baseline', y_pred_xgb, 'Greys'),
    ('XGB Balanced', y_pred_balanced, 'Blues'),
    ('XGB SMOTE', y_pred_smote, 'Greens'),
    ('Random Forest', y_pred_rf, 'Purples')
]

# Creamos la figura: 2 filas, 3 columnas
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle(f'Matrices de Confusión (Umbral = {threshold})', fontsize=16, fontweight='bold')

# "Aplanamos" la matriz de ejes para poder iterar fácilmente (de 2D a 1D)
axes_flat = axes.flatten()

for i, (name, y_pred_proba, cmap) in enumerate(models_cm):
    ax = axes_flat[i]

    # 1. Calcular predicción binaria y matriz
    y_pred_class = (y_pred_proba >= threshold).astype(int)
    cm = confusion_matrix(y_test, y_pred_class)

    # 2. Extraer valores
    tn, fp, fn, tp = cm.ravel()

    # 3. Graficar Heatmap
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap,
                xticklabels=['No Default', 'Default'],
                yticklabels=['No Default', 'Default'],
                ax=ax, cbar=False, annot_kws={"size": 12}) # cbar=False para limpiar, annot_kws para tamaño num

    # 4. Estética del subplot
    ax.set_title(f'{name}\nVN={tn} | FP={fp} | FN={fn} | VP={tp}',
                 fontsize=11, fontweight='bold')
    ax.set_ylabel('Etiqueta Real' if i % 3 == 0 else '') # Solo etiqueta Y en la primera columna
    ax.set_xlabel('Predicción')

# --- Manejo del espacio sobrante ---
# Como tenemos 5 modelos y 6 espacios, ocultamos el último (índice 5)
axes_flat[5].axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.95]) # Ajuste

In [ ]:
rf_balanced = RandomForestClassifier(
    n_estimators=300,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=0
)

rf_balanced.fit(X_train, y_train)
y_pred_rf = rf_balanced.predict_proba(X_test)[:, 1]

# --- Cálculo de Métricas ---
auc_rf = roc_auc_score(y_test, y_pred_rf)
gini_rf = 2 * auc_rf - 1

# Cálculo de KS y búsqueda del Umbral Óptimo
scores_rf_0 = y_pred_rf[y_test == 0]
scores_rf_1 = y_pred_rf[y_test == 1]
ks_rf_stat = stats.ks_2samp(scores_rf_0, scores_rf_1)
ks_rf = ks_rf_stat.statistic

# Encontrar el umbral donde el KS es máximo (opcional pero recomendado)
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, y_pred_rf)
ks_curve_rf = tpr_rf - fpr_rf
idx_max_ks_rf = np.argmax(ks_curve_rf)
best_thresh_rf = thresholds_rf[idx_max_ks_rf]

print(f"\nResultados Random Forest:")
print(f"AUC:  {auc_rf:.4f}")
print(f"Gini: {gini_rf:.4f}")
print(f"KS:   {ks_rf:.4f}")
print(f"Umbral Óptimo (KS): {best_thresh_rf:.4f}")

# Matriz con el umbral óptimo
y_pred_class_rf_opt = (y_pred_rf >= best_thresh_rf).astype(int)
print("Matriz de Confusión (Umbral Óptimo):")
print(confusion_matrix(y_test, y_pred_class_rf_opt))

In [ ]:
logit_balanced = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42
)

logit_balanced.fit(X_train_red_scaled, y_train)
y_pred_logit_balanced = logit_balanced.predict_proba(X_test_red_scaled)[:, 1]

# --- Cálculo de Métricas ---
auc_logit = roc_auc_score(y_test, y_pred_logit_balanced)
gini_logit = 2 * auc_logit - 1

# Cálculo de KS y búsqueda del Umbral Óptimo
scores_logit_0 = y_pred_logit_balanced[y_test == 0]
scores_logit_1 = y_pred_logit_balanced[y_test == 1]
ks_logit_stat = stats.ks_2samp(scores_logit_0, scores_logit_1)
ks_logit = ks_logit_stat.statistic

# Encontrar el umbral donde el KS es máximo
fpr_log, tpr_log, thresholds_log = roc_curve(y_test, y_pred_logit_balanced)
ks_curve_log = tpr_log - fpr_log
idx_max_ks_log = np.argmax(ks_curve_log)
best_thresh_log = thresholds_log[idx_max_ks_log]

print(f"\nResultados Logit Balanced:")
print(f"AUC:  {auc_logit:.4f}")
print(f"Gini: {gini_logit:.4f}")
print(f"KS:   {ks_logit:.4f}")
print(f"Umbral Óptimo (KS): {best_thresh_log:.4f}")

# Matriz con el umbral óptimo
y_pred_class_log_opt = (y_pred_logit_balanced >= best_thresh_log).astype(int)
print("Matriz de Confusión (Umbral Óptimo):")
print(confusion_matrix(y_test, y_pred_class_log_opt))


## 16. Selección del Modelo Ganador y Umbral Óptimo
Tras el análisis comparativo, se selecciona el modelo de **Regresión Logística Balanceada** debido a su equilibrio entre interpretabilidad, estabilidad y desempeño comercial. 

Se propone una regla de negocio basada en el umbral 0.64:
- Probabilidad >= 0.64: **Rechazar** (Alto Riesgo).
- Probabilidad < 0.64: **Aprobar** (Bajo Riesgo).

## 17. Análisis de Cosechas (Vintage Analysis)
Evaluamos el desempeño del modelo a través del tiempo, agrupando a los clientes por su mes de originación. Esto permite identificar si la calidad de la cartera está mejorando o empeorando.

In [ ]:
eval_df = apps_model.copy()

eval_df['pd'] = logit_balanced.predict_proba(
    scaler.transform(eval_df[good_vars])
)[:, 1]

# cohorte por mes de originación
eval_df['cohort'] = eval_df['orig_month'].dt.to_period('M')




In [ ]:
cohort_table = (
    eval_df
    .groupby('cohort')
    .agg(
        avg_pd=('pd', 'mean'),
        real_default=('default_12m', 'mean'),
        total_amount=('amount', 'sum'),
        n_clients=('customer_id', 'count')
    )
)

cohort_table


In [ ]:
early = cohort_table.head(2)
recent = cohort_table.tail(2)

print("Cosechas tempranas:\n", early)
print("\nCosechas recientes:\n", recent)


## 18. Análisis de Riesgo por Deciles
Agrupamos a la población en 10 partes iguales según su probabilidad de default estimada. Esto nos permite calcular la Pérdida Esperada por segmento.

In [ ]:
eval_df['pd_percentile'] = pd.qcut(eval_df['pd'], 10, labels=False)


In [ ]:
percentile_table = (
    eval_df
    .groupby('pd_percentile')
    .agg(
        avg_pd=('pd', 'mean'),
        real_default=('default_12m', 'mean'),
        total_amount=('amount', 'sum'),
        n_clients=('customer_id', 'count')
    )
)

percentile_table['expected_loss'] = (
    percentile_table['avg_pd'] * percentile_table['total_amount']
)

percentile_table


In [ ]:
coef = pd.Series(logit_balanced.coef_[0], index=good_vars).sort_values()
coef


In [ ]:
coef.tail(5)


## 19. Interpretación de Coeficientes y Casos Extremos
Revisamos qué variables están impulsando el riesgo en el modelo final y analizamos los casos individuales con mayor y menor probabilidad de incumplimiento.

In [ ]:
eval_df.sort_values('pd', ascending=False).head(5)


Clientes menos riesgosos:

In [ ]:
eval_df.sort_values('pd').head(5)


## 20. Análisis de Sensibilidad (Stress Test)
Simulamos un escenario de crisis económica donde la probabilidad de default se incrementa un 25%. Calculamos el impacto financiero que esto tendría en la pérdida esperada de la cartera.

In [47]:
# Escenario adverso: incremento del 25% en la PD
eval_df = apps_model.copy()

eval_df['pd_base'] = logit_balanced.predict_proba(
    scaler.transform(eval_df[good_vars])
)[:, 1]

eval_df['pd_adverse'] = (eval_df['pd_base'] * 1.25).clip(0, 1)


C:\Users\leo_m\.conda\envs\Prueba-Monex\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [48]:
# Pérdida esperada
eval_df['el_base'] = eval_df['pd_base'] * eval_df['amount']
eval_df['el_adverse'] = eval_df['pd_adverse'] * eval_df['amount']

# Resultados agregados
stress_summary = eval_df[['el_base', 'el_adverse']].sum()
stress_summary


el_base       6.258451e+08
el_adverse    7.820680e+08
dtype: float64

In [49]:
impact = (
    (stress_summary['el_adverse'] - stress_summary['el_base'])
    / stress_summary['el_base']
)

print(f"Incremento porcentual de la pérdida esperada bajo estrés: {impact:.2%}")


Incremento porcentual de la pérdida esperada bajo estrés: 24.96%


El mayor impacto está en los deciles altos de PD

Eso valida que el modelo ordena bien el riesgo

In [50]:
stress_by_decile = (
    eval_df
    .assign(pd_decile=pd.qcut(eval_df['pd_base'], 10, labels=False))
    .groupby('pd_decile')
    .agg(
        el_base=('el_base', 'sum'),
        el_adverse=('el_adverse', 'sum')
    )
)

stress_by_decile['impact_pct'] = (
    (stress_by_decile['el_adverse'] - stress_by_decile['el_base'])
    / stress_by_decile['el_base']
)

stress_by_decile


,el_base,el_adverse,impact_pct
pd_decile,,,
0,3.378799e+07,4.223498e+07,0.250000
1,4.468498e+07,5.585623e+07,0.250000
2,5.067698e+07,6.334622e+07,0.250000
3,5.773169e+07,7.216461e+07,0.250000
4,6.209572e+07,7.761965e+07,0.250000
5,6.230817e+07,7.788522e+07,0.250000
6,6.883597e+07,8.604496e+07,0.250000
7,7.494652e+07,9.368316e+07,0.250000
8,8.114781e+07,1.014348e+08,0.250000
